In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

%matplotlib inline

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
np.random.seed(0)

#Importando dados
data_teste = pd.read_csv("/kaggle/input/titanic/test.csv")
data_train = pd.read_csv("/kaggle/input/titanic/train.csv")

data_train.describe()

## *Analise Inicial do Dataset*

In [ ]:
data_train.head()

In [ ]:
data_teste.head()

In [ ]:
print (data_train.info())
print ()
print (data_train.info())

In [ ]:
print ('Train Columns: ', data_train.columns)

print ('\nTest Columns: ', data_teste.columns)

- Categorical Features: Sex, Survived, PClass, Embarked
- Numerical Features: Age (Continua), Fare (Continua), SibSp (Discreta), Parch (Discreta)
- Alphanumeric: Cabin, Ticket

In [ ]:
data_train.isnull().sum() # Analisando as colunas com valores faltantes

In [ ]:
data_teste.isnull().sum() # Analisando as colunas com valores faltantes

In [ ]:
train_data = data_train.copy()
teste_data = data_teste.copy() # Copia para n alterar o banco de dados original
cols_drop = ['Name', 'Cabin', 'Ticket']
train = train_data.drop(cols_drop, axis = 1)
teste = teste_data.drop(cols_drop, axis = 1)

## *Exibindo Graficos de Sobrevivencia*

In [ ]:
# Criando uma copia do DataSet para plotar graficos
graph_train = data_train.copy()

In [ ]:
# Separando novamente a coluna Age em grupos, porem utilizando strings
bins = [0, 16, 32, 50, 64, np.inf]
labels = ['Criança', 'Jovem', 'Adulto', 'Senhor', 'Idoso']
graph_train['AgeGroup'] = pd.cut(graph_train["Age"], bins, labels = labels)

plt.title('Sobrevivencia por Grupo de Idade')
sns.barplot(x="AgeGroup", y="Survived", data=graph_train)

In [ ]:
# Separando novamente a coluna Fare em grupos, porem utilizando strings
graph_train['FareGroup'] = pd.qcut(graph_train["Fare"], q=6, labels=['Muito Baixo', 'Baixo', 'Médio-Baixo', 'Médio', 'Alto', 'Muito Alto'])

plt.title('Sobrevivencia por Tarifa')
sns.barplot(x="FareGroup", y="Survived", data=graph_train)

In [ ]:
sns.barplot(x="Parch", y="Survived", data=graph_train)

In [ ]:
sns.barplot(x="SibSp", y="Survived", data=graph_train)

In [ ]:
sns.barplot(x="Sex", y="Survived", data=graph_train)

## *Treinando o modelo*

In [ ]:
def avaliate_pipes(data, encoder, model, numerical_imputer, categorical_imputer):
    
    # Definindo Feature e Target
    X = data.drop('Survived', axis=1) # Features
    y = data['Survived'] # Target

    # Dividindo os dados
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Classificando as features em num e cat
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns  
    categorical_features = X.select_dtypes(include=['object']).columns

    # Aplicando Encoder nas features categóricas
    if encoder == 'one-hot':
        cat_encoder = OneHotEncoder(handle_unknown='ignore')
    elif encoder == 'ordinal':
        cat_encoder = OrdinalEncoder()
    elif encoder == 'label':
        for col in categorical_features:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
        cat_encoder = 'passthrough'
    else:
        raise ValueError('Encoder não reconhecido')

    # Tratando dados NaN com imputers
    numerical_transformer = SimpleImputer(strategy= numerical_imputer if numerical_imputer != 'zero' else 'constant', fill_value=0)
    categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy= categorical_imputer )), # Trata das colunas categoricas utilizando um SimpleImputer para trocar os dados NaN 
    ( encoder, cat_encoder) # Utiliza o Encoder escolhido para transformar as colunas categoricas em Numericas
])
  
    # Criando um preprocessor
    preprocessor = ColumnTransformer(    # Processo anterior a aplicação do modelo, tratamento de dados, permite transformar colunas de forma independente
        transformers=[                   # transformers = ['nome', 'transformador', 'coluna']
            ('num', numerical_transformer, numerical_features),   # tranforma as numerical features com o numerical transformer
            ('cat', categorical_transformer, categorical_features)  # tranforma as categorical features com o categorical transformer
        ])
    
    # Criando um pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor), # Definindo passos do pipe, 1º passa pelo preprocessor cujo vai transformar as colunas 
        ('model', model)                # Utilizando os Imputers para preencher os NaN's e realizar o encoder das categoricas
    ])                                  # 2º Passa os dados ja processados pelo model fazendo seu treino e entregando ao final seu Score.

    # Treinando o modelo
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    
    return pipeline, score*100

In [ ]:
model = RandomForestClassifier()
max_score = 0
for encoder in ['ordinal', 'one-hot']:
    print()
    for numerical_imputer in ['mean', 'median', 'constant', 'zero']:
        for categorical_imputer in ['most_frequent', 'constant']:
            pipeline, score = avaliate_pipes(train, encoder, model, numerical_imputer, categorical_imputer)
            print(f'Score: {score:.4}%, Model: {model}, Encoder: {encoder}, Numerical Imputer: {numerical_imputer}, Categorical Imputer: {categorical_imputer}')
            if score >= max_score:
                max_score = score
                B_encoder = encoder
                B_Num = numerical_imputer
                B_Cat = categorical_imputer
            else :max_score
            
print()

print(f'Melhor score: {max_score:.4}%')

In [ ]:
print(B_encoder, B_Num, B_Cat)

In [ ]:
pipeline, score = avaliate_pipes(train, B_encoder, model, B_Num, B_Cat)

## *Gerando modelo para competição*

In [ ]:
X_teste = teste
teste_ids = teste['PassengerId']
predictions = pipeline.predict(X_teste)
submission = pd.DataFrame({"PassengerId": teste_ids, "Survived": predictions})
submission.to_csv("submission.csv", index=False)